In [3]:
from pywaterinfo import Waterinfo
import os
import pandas as pd
import numpy as np
import schedule
import datetime
import time

In [4]:
vmm_group_list = pd.DataFrame()
hic_group_list = pd.DataFrame()
vmm_group_id = []
hic_group_id = []
vmm_ts_id = pd.DataFrame()
hic_ts_id = pd.DataFrame()
vmm_ts_values = pd.DataFrame()
hic_ts_values = pd.DataFrame()

In [5]:
vmm = Waterinfo("vmm") #flanders enviroment agency
hic = Waterinfo("hic") #flanders hydraulics research

In [6]:
def fetch_vmm_hic_group_list():
    global vmm_group_list,hic_group_list,vmm_group_id,hic_group_id
    vmm_group_list = vmm.get_group_list()
    hic_group_list = hic.get_group_list()
    vmm_group_list = vmm_group_list[(vmm_group_list['group_type'] == 'timeseries') | (vmm_group_list['group_type'] == 'timeseries_dynamic')]
    hic_group_list = hic_group_list[(hic_group_list['group_type'] == 'timeseries') | (hic_group_list['group_type'] == 'timeseries_dynamic')]



    vmm_group_list.to_csv('vmm_group_list.csv',index=False,mode='w')
    hic_group_list.to_csv('hic_group_list.csv',index=False,mode='w')

In [7]:
fetch_vmm_hic_group_list()

In [8]:
def fetch_vmm_hic_group_id():
    global vmm_group_id,hic_group_id,vmm_ts_id,hic_ts_id
    counter = 0;
    vmm_group_list = pd.read_csv('vmm_group_list.csv')
    hic_group_list = pd.read_csv('hic_group_list.csv')

    vmm_group_id = vmm_group_list["group_id"]
    hic_group_id = hic_group_list["group_id"]
    
    for i in vmm_group_id:
        ts_data = vmm.get_timeseries_value_layer(timeseriesgroup_id=i)
        counter=counter+1
        #print(i)
        if ts_data is not None and not ts_data.empty:
            ts_data = ts_data.copy()  # Make a copy to avoid modifying the original DataFrame
            ts_data['group_id'] = i 
            vmm_ts_id = pd.concat([vmm_ts_id,ts_data], ignore_index=True)
        #if counter==10:
            #break
            
            
    vmm_ts_id.to_csv('vmm_ts_id.csv',index=False,header=True,mode='w')

    
    for i in hic_group_id:
        ts_data = hic.get_timeseries_value_layer(timeseriesgroup_id = i)
        if ts_data is not None and not ts_data.empty:
            ts_data = ts_data.copy()  # Make a copy to avoid modifying the original DataFrame
            ts_data['group_id'] = i 
            hic_ts_id = pd.concat([hic_ts_id,ts_data], ignore_index=True)
            
    hic_ts_id.to_csv('hic_ts_id.csv',index=False,header=True,mode='w')

In [9]:
fetch_vmm_hic_group_id()

/tmp/ipykernel_7111/3910304129.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vmm_ts_id = pd.concat([vmm_ts_id,ts_data], ignore_index=True)
/tmp/ipykernel_7111/3910304129.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vmm_ts_id = pd.concat([vmm_ts_id,ts_data], ignore_index=True)
/tmp/ipykernel_7111/3910304129.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA column

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [12]:
def get_vmm_hic_timeseries():
    global vmm_group_id,hic_group_id,vmm_ts_id,hic_ts_id,vmm_ts_values,hic_ts_values
    
    for index, row in vmm_ts_id.iterrows():
        #print(vmm.get_timeseries_values(row['ts_id'], period="P5Y"))
        vmm_ts_values = pd.concat([vmm_ts_values,vmm.get_timeseries_values(row['ts_id'], period="P1D")], ignore_index=True)
        
    vmm_ts_values.to_csv('vmm_ts_values.csv',index=False,header=True,mode='w')
    
    for index, row in hic_ts_id.iterrows():
        hic_ts_values = pd.concat([hic_ts_values,hic.get_timeseries_values(row['ts_id'], period="P1D")], ignore_index=True)
        
    hic_ts_values.to_csv('hic_ts_values.csv',index=False,header=True,mode='w')




In [13]:
get_vmm_hic_timeseries()

/tmp/ipykernel_7111/801115914.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vmm_ts_values = pd.concat([vmm_ts_values,vmm.get_timeseries_values(row['ts_id'], period="P1D")], ignore_index=True)
/tmp/ipykernel_7111/801115914.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vmm_ts_values = pd.concat([vmm_ts_values,vmm.get_timeseries_values(row['ts_id'], period="P1D")], ignore_index=True)
/tmp/ipykernel_7111/801115914.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or

KiwisException: Waterinfo call returned 500 error with the message b'{"type": "error", "code": "DatasourceError", "message": "Error retrieving WAPI response: {\\"type\\":\\"de.kisters.wiski7.wiskidataaccess.WiskiDataException\\",\\"message\\":\\"Illegal time interval: from=1949-12-31T23:00:00.000Z, until=1949-12-31T23:00:00.000Z, leftClosed=true, rightClosed=false.\\",\\"stacktrace\\":\\"de.kisters.wiski7.wiskidataaccess.WiskiDataException: Illegal time interval: from=1949-12-31T23:00:00.000Z, until=1949-12-31T23:00:00.000Z, leftClosed=true, rightClosed=false.\\\\r\\\\n\\\\tat de.kisters.wiski7.wiskidataaccess.impl.W7ExtTimeSeries.getReleaseStates(W7ExtTimeSeries.java:953)\\\\r\\\\n\\\\tat de.kisters.wiski7.wiskidataaccess.rest.core.TimeSeries.getReleases(TimeSeries.java:2791)\\\\r\\\\n\\\\tat jdk.internal.reflect.GeneratedMethodAccessor184.invoke(Unknown Source)\\\\r\\\\n\\\\tat java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)\\\

In [14]:
df = hic.get_timeseries_values(ts_id="39496010", period="P1D")
print(df)

                    Timestamp  Value  Interpolation Type  Quality Code  \
0   2025-03-23 16:20:00+00:00    723                 102           111   
1   2025-03-23 16:25:00+00:00    723                 102           111   
2   2025-03-23 16:30:00+00:00    723                 102           111   
3   2025-03-23 16:35:00+00:00    723                 102           111   
4   2025-03-23 16:40:00+00:00    723                 102           111   
..                        ...    ...                 ...           ...   
282 2025-03-24 15:50:00+00:00    731                 102           111   
283 2025-03-24 15:55:00+00:00    730                 102           111   
284 2025-03-24 16:00:00+00:00    731                 102           111   
285 2025-03-24 16:05:00+00:00    732                 102           111   
286 2025-03-24 16:10:00+00:00    732                 102           111   

    Quality Code Name                           Quality Code Description  \
0              A_C1_1  Good quality